In [1]:
import cv2
import numpy as np
import math
import time
import mediapipe as mp

In [8]:
class HandDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, minTrackCon=0.5):
    
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.minTrackCon = minTrackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(static_image_mode=self.mode, max_num_hands=self.maxHands,
                                        min_detection_confidence=self.detectionCon,
                                        min_tracking_confidence=self.minTrackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]
        self.fingers = []
        self.lmList = []
        
    def findHands(self, img, draw=True, flipType=True):
     
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            self.results = self.hands.process(imgRGB)
            allHands = []
            h, w, c = img.shape
            if self.results.multi_hand_landmarks:
                for handType, handLms in zip(self.results.multi_handedness, self.results.multi_hand_landmarks):
                    myHand = {}
                    ## lmList
                    mylmList = []
                    xList = []
                    yList = []
                    for id, lm in enumerate(handLms.landmark):
                        px, py, pz = int(lm.x * w), int(lm.y * h), int(lm.z * w)
                        mylmList.append([px, py, pz])
                        xList.append(px)
                        yList.append(py)

                    ## bbox
                    xmin, xmax = min(xList), max(xList)
                    ymin, ymax = min(yList), max(yList)
                    boxW, boxH = xmax - xmin, ymax - ymin
                    bbox = xmin, ymin, boxW, boxH
                    cx, cy = bbox[0] + (bbox[2] // 2), \
                             bbox[1] + (bbox[3] // 2)

                    myHand["lmList"] = mylmList
                    myHand["bbox"] = bbox
                    myHand["center"] = (cx, cy)

                    if flipType:
                        if handType.classification[0].label == "Right":
                            myHand["type"] = "Right"
                        else:
                            myHand["type"] = "Left"
                    else:
                        myHand["type"] = handType.classification[0].label
                    allHands.append(myHand)

                    ## draw
                    if draw:
                        self.mpDraw.draw_landmarks(img, handLms,
                                                   self.mpHands.HAND_CONNECTIONS)
                        #cv2.rectangle(img, (bbox[0] - 20, bbox[1] - 20),
                         #             (bbox[0] + bbox[2] + 20, bbox[1] + bbox[3] + 20),
                          #            (255, 0, 255), 2)
                        #cv2.putText(img, myHand["type"], (bbox[0] - 30, bbox[1] - 30), cv2.FONT_HERSHEY_PLAIN,
                         #           2, (255, 0, 255), 2)
            if draw:
                return allHands, img
            else:
                return allHands

        
    def fingersUp(self, myHand):
        
        myHandType = myHand["type"]
        myLmList = myHand["lmList"]
        if self.results.multi_hand_landmarks:
            fingers = []
            # Thumb
            if myHandType == "Right":
                if myLmList[self.tipIds[0]][0] > myLmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
            else:
                if myLmList[self.tipIds[0]][0] < myLmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            # 4 Fingers
            for id in range(1, 5):
                if myLmList[self.tipIds[id]][1] < myLmList[self.tipIds[id] - 2][1]:
                    fingers.append(1)
                else:
                    fingers.append(0)
        return fingers

    def findDistance(self, p1, p2, img=None):
        
        x1, y1 = p1
        x2, y2 = p2
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        length = math.hypot(x2 - x1, y2 - y1)
        info = (x1, y1, x2, y2, cx, cy)
        if img is not None:
            cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
            cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
            return length, info, img
        else:
            return length, info

        
def main():
    cap = cv2.VideoCapture(0)
    detector = HandDetector(detectionCon=0.8, maxHands=2)
    while True:
        # Get image frame
        
        success, img = cap.read()
        # Find the hand and its landmarks
        hands, img = detector.findHands(img)  # with draw
        # hands = detector.findHands(img, draw=False)  # without draw

        if hands:
            # Hand 1
            hand1 = hands[0]
            print(hand1)
            lmList1 = hand1["lmList"]  # List of 21 Landmark points
            bbox1 = hand1["bbox"]  # Bounding box info x,y,w,h
            centerPoint1 = hand1['center']  # center of the hand cx,cy
            handType1 = hand1["type"]  # Handtype Left or Right

            fingers1 = detector.fingersUp(hand1)

            if len(hands) == 2:
                # Hand 2
                hand2 = hands[1]
                lmList2 = hand2["lmList"]  # List of 21 Landmark points
                bbox2 = hand2["bbox"]  # Bounding box info x,y,w,h
                centerPoint2 = hand2['center']  # center of the hand cx,cy
                handType2 = hand2["type"]  # Hand Type "Left" or "Right"

                fingers2 = detector.fingersUp(hand2)

                # Find Distance between two Landmarks. Could be same hand or different hands
                length, info, img = detector.findDistance(lmList1[8][0:2], lmList2[8][0:2], img)  # with draw
                # length, info = detector.findDistance(lmList1[8], lmList2[8])  # with draw
        # Display
        cv2.imshow("Image", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

{'lmList': [[194, 462, 0], [244, 453, -29], [285, 420, -45], [314, 384, -57], [345, 357, -71], [257, 346, -36], [285, 295, -62], [306, 262, -81], [325, 237, -93], [224, 335, -37], [249, 276, -58], [268, 237, -75], [285, 205, -86], [189, 339, -41], [207, 279, -62], [220, 241, -77], [236, 210, -86], [152, 355, -46], [147, 312, -68], [151, 285, -78], [160, 262, -83]], 'bbox': (147, 205, 198, 257), 'center': (246, 333), 'type': 'Left'}
{'lmList': [[195, 466, 0], [245, 455, -28], [286, 422, -43], [315, 386, -56], [345, 357, -69], [257, 344, -31], [286, 294, -54], [306, 263, -72], [324, 238, -84], [225, 333, -31], [250, 273, -50], [269, 235, -65], [285, 204, -76], [189, 337, -35], [208, 275, -54], [223, 238, -68], [239, 208, -76], [152, 354, -41], [148, 310, -60], [153, 282, -68], [162, 260, -72]], 'bbox': (148, 204, 197, 262), 'center': (246, 335), 'type': 'Left'}
{'lmList': [[198, 440, 0], [250, 421, -21], [287, 378, -31], [308, 336, -39], [334, 305, -47], [260, 309, -16], [283, 256, -35],

{'lmList': [[170, 430, 0], [220, 402, -16], [250, 354, -21], [262, 309, -24], [279, 275, -28], [217, 292, 0], [231, 243, -11], [241, 215, -23], [251, 192, -31], [188, 285, -1], [197, 229, -9], [205, 194, -19], [213, 167, -26], [159, 289, -6], [164, 236, -16], [171, 203, -25], [182, 176, -31], [130, 303, -12], [122, 266, -23], [125, 243, -28], [132, 222, -31]], 'bbox': (122, 167, 157, 263), 'center': (200, 298), 'type': 'Left'}
{'lmList': [[172, 431, 0], [222, 403, -16], [252, 356, -21], [264, 312, -26], [280, 277, -29], [218, 293, 0], [233, 244, -12], [244, 216, -24], [254, 193, -32], [189, 286, -2], [199, 230, -10], [208, 196, -20], [216, 168, -28], [162, 290, -7], [167, 237, -17], [175, 205, -26], [186, 177, -32], [133, 305, -13], [125, 267, -25], [128, 244, -30], [136, 223, -33]], 'bbox': (125, 168, 155, 263), 'center': (202, 299), 'type': 'Left'}
{'lmList': [[177, 432, 0], [227, 406, -16], [258, 358, -20], [272, 315, -24], [289, 281, -27], [226, 295, -1], [241, 246, -13], [251, 218